<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/projection/SD_unet_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

%cd drive/My\ Drive/


Mounted at /content/drive/
/content/drive/My Drive


# LOAD MODELS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sqlite3

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *
from utils.unet import *

# Load data

In [3]:
data_all = pd.read_table('./data/gps+5-30s.csv', sep = ',')
check = pd.read_table('./data/check_gps+5-30s.csv', sep = ',')

# SD 5s
## data

In [4]:
select = check.trip[(check.species == 'SD') & (check.res == 5)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
1264,F0417_11_SD_T1,2017-04-25 14:23:05,-32.389526,-3.818225,0.0,10.351890,4.582113,0,-2.059880,1.253909,0.469376,0.996804,7.988774e-02
1265,F0417_11_SD_T1,2017-04-25 14:23:10,-32.389126,-3.818093,0.0,9.358567,-25.838874,0,-2.056048,1.256616,0.424337,0.900023,-4.358418e-01
1266,F0417_11_SD_T1,2017-04-25 14:23:15,-32.388988,-3.817499,0.0,13.574546,-58.648103,0,-2.054726,1.268801,0.615498,0.520293,-8.539879e-01
1267,F0417_11_SD_T1,2017-04-25 14:23:20,-32.388897,-3.816828,0.0,15.074257,-5.344776,0,-2.053854,1.282565,0.683498,0.995652,-9.314870e-02
1268,F0417_11_SD_T1,2017-04-25 14:23:25,-32.388702,-3.816197,0.0,14.700266,9.430723,0,-2.051985,1.295509,0.666540,0.986484,1.638550e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248978,F0915_1_SD_T9,2015-09-26 15:27:53,-32.422874,-3.872315,0.0,20.191487,0.000021,0,1.272149,2.868065,0.870392,1.000000,3.725678e-07
248979,F0915_1_SD_T9,2015-09-26 15:27:58,-32.422468,-3.871368,0.0,22.932451,-1.903076,0,1.494617,3.109940,0.988547,0.999448,-3.320883e-02
248980,F0915_1_SD_T9,2015-09-26 15:28:03,-32.422062,-3.870421,0.0,22.932455,0.000023,0,1.717085,3.351814,0.988547,1.000000,4.045372e-07
248981,F0915_1_SD_T9,2015-09-26 15:28:08,-32.421869,-3.870170,0.0,7.039456,14.287203,0,1.822638,3.415922,0.303449,0.969071,2.467826e-01


In [5]:
model = torch.load('SV_unet_matrixlonlatcov_5s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [6]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F0417_11_SD_T1
F0417_11_SD_T2
F0417_3_SD_T1
F0417_5_SD_T1
F0417_7_SD_T1
F0417_9_SD_T1
F0915_1_SD_T1
F0915_1_SD_T2
F0915_1_SD_T3
F0915_1_SD_T4
F0915_1_SD_T5
F0915_1_SD_T6
F0915_1_SD_T7
F0915_1_SD_T8
F0915_1_SD_T9


In [7]:
data.to_csv('SD_5s.csv', index = False)

# SD 15s
## data

In [8]:
select = check.trip[(check.species == 'SD') & (check.res == 15)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
2,F0417_1_SD_T1,2017-04-22 16:42:54,-32.418946,-3.875840,0.0,11.859596,-21.002254,0,-0.540653,1.334173,0.645704,0.933566,-0.358405
3,F0417_1_SD_T1,2017-04-22 16:43:09,-32.417819,-3.877086,0.0,12.451647,-7.037871,0,-0.514006,1.310184,0.678779,0.992465,-0.122525
4,F0417_1_SD_T1,2017-04-22 16:43:24,-32.417688,-3.878604,0.0,11.305211,37.147540,0,-0.510902,1.280948,0.614732,0.797083,0.603870
5,F0417_1_SD_T1,2017-04-22 16:43:39,-32.417353,-3.879996,0.0,10.627715,-8.526295,0,-0.503003,1.254119,0.576883,0.988948,-0.148263
6,F0417_1_SD_T1,2017-04-22 16:43:54,-32.417096,-3.881330,0.0,10.078826,2.549302,0,-0.496912,1.228430,0.546219,0.999010,0.044479
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376426,F0916_8_SD_T1,2016-09-17 13:47:32,-31.767923,-3.959683,0.0,11.792293,-13.616328,0,-1.350309,-1.449720,0.604929,0.971894,-0.235419
376427,F0916_8_SD_T1,2016-09-17 13:47:47,-31.768166,-3.960852,0.0,8.861321,-34.952377,0,-1.351163,-1.456297,0.454574,0.819629,-0.572895
376428,F0916_8_SD_T1,2016-09-17 13:48:02,-31.768394,-3.962101,0.0,9.418448,-1.383647,0,-1.351965,-1.463320,0.483154,0.999708,-0.024147
376429,F0916_8_SD_T1,2016-09-17 13:48:17,-31.768583,-3.962924,0.0,6.265243,2.628907,0,-1.352632,-1.467948,0.321399,0.998948,0.045867


In [9]:
model = torch.load('SV_unet_matrixlonlatcov_15s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [10]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F0417_1_SD_T1
F0417_10_SD_T1
F0417_12_SD_T1
F0417_13_SD_T1
F0417_13_SD_T2
F0417_13_SD_T3
F0417_14_SD_T1
F0417_14_SD_T2
F0417_15_SD_T1
F0417_15_SD_T2
F0417_16_SD_T1
F0417_16_SD_T2
F0417_16_SD_T3
F0417_17_SD_T1
F0417_17_SD_T2
F0417_18_SD_T1
F0417_18_SD_T2
F0417_18_SD_T3
F0417_19_SD_T1
F0417_19_SD_T2
F0417_19_SD_T3
F0417_2_SD_T1
F0417_20_SD_T1
F0417_20_SD_T2
F0417_21_SD_T1
F0417_21_SD_T2
F0417_21_SD_T3
F0417_21_SD_T4
F0417_22_SD_T1
F0417_22_SD_T2
F0417_24_SD_T1
F0417_24_SD_T2
F0417_24_SD_T3
F0417_24_SD_T4
F0417_25_SD_T1
F0417_26_SD_T1
F0417_27_SD_T1
F0417_28_SD_T1
F0417_28_SD_T2
F0417_29_SD_T1
F0417_30_SD_T1
F0417_6_SD_T1
F0417_8_SD_T1
F0418_10_SD_T1
F0418_11_SD_T1
F0418_11_SD_T2
F0418_11_SD_T3
F0418_12_SD_T1
F0418_12_SD_T2
F0418_14_SD_T1
F0418_14_SD_T2
F0418_15_SD_T1
F0418_22_SD_T1
F0418_23_SD_T1
F0418_23_SD_T2
F0418_24_SD_T1
F0418_24_SD_T2
F0418_25_SD_T1
F0418_25_SD_T2
F0418_26_SD_T1
F0418_26_SD_T2
F0418_30_SD_T1
F0418_30_SD_T2
F0418_30_SD_T3
F0418_30_SD_T4
F0418_31_SD_T1
F0418_31_SD_T2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


F0419_6_SD_T2
F0419_6_SD_T3
F0419_7_SD_T1
F0419_7_SD_T2
F0419_7_SD_T3
F0916_8_SD_T1


In [11]:
data.to_csv('SD_15s.csv', index = False)

# SD 30s
## data

In [12]:
select = check.trip[(check.species == 'SD') & (check.res == 30)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
145103,F0418_32_SD_T2,2018-04-25 06:07:50,-32.384932,-3.814322,0.0,12.494994,14.168260,0,-1.630901,1.016214,0.589121,0.969581,0.244770
145104,F0418_32_SD_T2,2018-04-25 06:08:20,-32.381637,-3.813599,0.0,12.492593,12.721614,0,-1.603449,1.027805,0.589007,0.975452,0.220214
145105,F0418_32_SD_T2,2018-04-25 06:08:50,-32.378399,-3.813556,0.0,11.989150,11.645721,0,-1.576476,1.028493,0.565226,0.979414,0.201860
145106,F0418_32_SD_T2,2018-04-25 06:09:20,-32.375278,-3.813707,0.0,11.567277,3.540646,0,-1.550481,1.026072,0.545299,0.998091,0.061757
145107,F0418_32_SD_T2,2018-04-25 06:09:50,-32.372065,-3.812919,0.0,12.250076,-16.575779,0,-1.523714,1.038687,0.577552,0.958443,-0.285283
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238391,F0419_9_SD_T3,2019-04-21 14:35:55,-31.962580,-3.519475,0.0,0.433256,6.944997,0,0.879874,0.654825,0.026370,0.992663,0.120916
238392,F0419_9_SD_T3,2019-04-21 14:36:25,-31.962788,-3.519439,0.0,0.780249,0.479404,0,0.878292,0.655181,0.047490,0.999965,0.008367
238393,F0419_9_SD_T3,2019-04-21 14:36:55,-31.963046,-3.519406,0.0,0.963976,-2.645391,0,0.876325,0.655505,0.058672,0.998934,-0.046154
238394,F0419_9_SD_T3,2019-04-21 14:37:25,-31.963135,-3.519404,0.0,0.330480,-5.958953,0,0.875645,0.655525,0.020115,0.994597,-0.103816


In [13]:
model = torch.load('SV_unet_matrixlonlatcov_30s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [14]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F0418_32_SD_T2
F0419_8_SD_T1
F0419_8_SD_T2
F0419_9_SD_T1
F0419_9_SD_T2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


F0419_9_SD_T3


In [15]:
data.to_csv('SD_30s.csv', index = False)